## Will Paz

In [1]:
import numpy as np

In [2]:
def rayleigh(A,x):
    """
    Applies the Rayleigh quotient
    Inputs: The matrix A and the vector x
    Output: The Rayleigh quotient value
    """
    top = np.dot(np.dot(A, x), x)
    bottom = np.dot(x, x) 
    r = top / bottom 
    return r

In [3]:
def algo1(A,steps):
    """
    Runs the first algorithm and returns the "best" eigenvector and eigenvalue for A in the number of given steps
    Inputs: The matrix A (symmetric) and the number of desired steps
    Output: The "best" eigenvector and value in the steps given
    """
    n,m = A.shape
    e_val_approx = np.zeros(steps+1)
    e_vec_approx = np.zeros((n,steps+1))
    
    v = np.random.rand(1,n)
    
    e_vec_approx[:,0] = v/np.linalg.norm(v)
    e_val_approx[0] = 0
    
    for k in range(1,steps+1):
        e_vec_approx[:,k] = (A @ e_vec_approx[:,k-1]) / np.linalg.norm((A @ e_vec_approx[:,k-1]))
        e_val_approx[k] = rayleigh(A,e_vec_approx[:,k])
    
    return e_vec_approx, e_val_approx

In [4]:
def test1(n,steps,error, stretch=0):
    """
    Tests the third algorithm
    Inputs: n, size of the matrix, number of desired steps, desired error, and stretch
    Output: 
    """
    A = np.random.randn(n,n)
    B = A @ A.T
    
    evals, evecs = np.linalg.eigh(B)
    
    index_eigh = np.argmax(evals)
    q = np.absolute(evecs[:,index_eigh])
    P = q @ q.T
    B_update = B + stretch * P
    
    def cvalind(C,error,steps):
        cvecs, cvals = algo1(C,steps)
        l = cvals[steps]
        
        k = 0
        
        while k <= steps:
            if np.abs(l - cvals[k]) < error:
                return k
            else:
                k = k + 1
        
        return np.inf
        
    def cvecind(C,error,steps):
        cvecs, cvals = algo1(C,steps)
        q = cvecs[:,steps]
        
        k = 0
        
        while k <= steps:
            if np.linalg.norm(q - cvecs[:,k]) < error or np.linalg.norm((-q) - cvecs[:,k]) < error:
                return k
            else:
                k = k + 1
        
        return np.inf
                
    B_val = cvalind(B,error,steps)
    B_vec = cvecind(B,error,steps)
    B_up_val = cvalind(B_update,error,steps)
    B_up_vec = cvecind(B_update,error,steps)
        
    return print(B_val,B_vec,B_up_val,B_up_vec)

In [5]:
def algo3(A,steps):
    """
    Runs the third algorithm and returns the "best" eigenvector and eigenvalue for A in the number of given steps
    Inputs: The matrix A (symmetric) and the number of desired steps
    Output: The "best" eigenvector and value in the steps given
    """
    n,_ = A.shape
    e_val_approx = np.zeros(steps + 1)
    e_vec_approx = np.zeros((n, steps + 1))
    
    v = np.random.rand(1,n)
    
    e_vec_approx[:, 0] = v / np.linalg.norm(v)
    e_val_approx[0] = rayleigh(A, e_vec_approx[:, 0])  
    
    for k in range(1, steps + 1):
        B = np.linalg.solve((A - e_val_approx[k - 1]),e_vec_approx[:,k-1])     
        e_vec_approx[:, k] = B / np.linalg.norm(B)
        e_val_approx[k] = rayleigh(A, e_vec_approx[:,k])  
        
    return e_vec_approx, e_val_approx

In [6]:
def test3(n,steps):
    """
    Tests the third algorithm
    Inputs: n, size of the matrix, and number of desired steps
    Output: Return a steps length vector where the ith entry is ||Ax[i] - y[i]x[i]||, where x[i] is the 
    ith eigenvector guess and y[i] is the ith eigenvalue guess
    """
    A = np.random.randn(n,n)
    B = A @ A.T
    
    x,y = algo3(B,steps)
    
    guess = np.zeros(steps+1)
    
    for i in range(steps+1):
        guess[i] = np.linalg.norm(np.dot(B,x[:,i]) - np.dot(y[i],x[:,i]))
    return guess